# ChatBot

## Data収集
日本語の学習用データを用意する。
* [開発データ・評価データ - 対話破綻検出チャレンジ2](https://sites.google.com/site/dialoguebreakdowndetection2/downloads)

In [1]:
import sys
import os
import json
import pickle

import numpy as np
import MeCab

In [2]:
def load_json(filepath):
    fj = open(filepath,'r')
    json_data = json.load(fj)
    fj.close()
    return json_data

In [3]:
def output(data, mode):
    for i in range(len(data['turns'])):
        if mode == "U" and data['turns'][i]['speaker'] == mode:
            print(data['turns'][i]['utterance'])
        elif mode == "S" and data['turns'][i]['speaker'] == mode and i != 0:
            print(data['turns'][i]['utterance'])
        else:
            continue

In [4]:
def output_with_pair(data):
    for i in range(len(data['turns'])):
        print(data['turns'][i]["speaker"] + ":" + data['turns'][i]['utterance'])

In [5]:
data = load_json("DBDC2_dev/IRS/1471400435.log.json")

In [6]:
output(data, "U")

オリンピックが気になりますね
高校野球も面白いですね
子供のころ友達と遊びでやった程度です
バスケットも人気高いですよね
今はどんなスポーツをやってるんですか？
映像といえばシン・ゴジラ観ましたか？
洋画も好きでたまに見ますよ
おススメの映画はありますか？
はい、気になる作品が上映されたら観に行くようにしています
世界で一人だけになったらとても寂しいですね


In [7]:
output(data, "S")

旬ですねぇ。自分もオリンピック書いたし。
今までにプレイしたことはありますか？
私は今はやってないですけど、高校までバスケをやってました。
いい所だよ！
すっっごい泣けますよ。後半特にあと、映像がやたらきれいでそれだけで観る価値あります。
洋画は好きですか
同じですね
よく映画を見に行くのですか。
人類に一人だけ残されたら・・・・子孫も残せないわけだからあまり生きてても意味がないような気がするんですけど。
そうですね、人とわいわいやるのも嫌いではないのですが、私はどちらかというと一人で何かに集中することのほうが好きなので、漫画もその一つです。


In [8]:
output_with_pair(data)

S:こんにちは。最近気になることとか、はまっていることとかありますか？
U:オリンピックが気になりますね
S:旬ですねぇ。自分もオリンピック書いたし。
U:高校野球も面白いですね
S:今までにプレイしたことはありますか？
U:子供のころ友達と遊びでやった程度です
S:私は今はやってないですけど、高校までバスケをやってました。
U:バスケットも人気高いですよね
S:いい所だよ！
U:今はどんなスポーツをやってるんですか？
S:すっっごい泣けますよ。後半特にあと、映像がやたらきれいでそれだけで観る価値あります。
U:映像といえばシン・ゴジラ観ましたか？
S:洋画は好きですか
U:洋画も好きでたまに見ますよ
S:同じですね
U:おススメの映画はありますか？
S:よく映画を見に行くのですか。
U:はい、気になる作品が上映されたら観に行くようにしています
S:人類に一人だけ残されたら・・・・子孫も残せないわけだからあまり生きてても意味がないような気がするんですけど。
U:世界で一人だけになったらとても寂しいですね
S:そうですね、人とわいわいやるのも嫌いではないのですが、私はどちらかというと一人で何かに集中することのほうが好きなので、漫画もその一つです。


### Mecab で 文章を分解する
* [Ubuntu 16.04.1 LTSにPython 3(Anaconda)とMeCabをインストールする : 二日坊主な私](http://blueskydb.blog.jp/archives/67055421.html)

In [9]:
m = MeCab.Tagger("-Owakati")
def get_text(data):
    user = []
    system = []
    for i in range(len(data['turns'])):
        if data['turns'][i]['speaker'] == "U":
            user.append(m.parse(data['turns'][i]['utterance']))
        elif data['turns'][i]['speaker'] == "S" and i != 0:
            system.append(m.parse(data['turns'][i]['utterance']))
        else:
            continue
    return user, system

### データをすべて読み込み

In [10]:
source_text = []
target_text = []

import glob
for path in glob.glob('*/*/*.log.json'):
    data = load_json(path)
    user, system = get_text(data)
    source_text.extend(user)
    target_text.extend(system)

In [11]:
len(source_text), len(target_text)

(4000, 4000)

### Data収集その2

In [12]:
source_text2 = []
target_text2 = []
m = MeCab.Tagger("-Owakati")

with open("sequence.txt",'r') as f:
    for row in f:
        if row.startswith("input:"):
            data = row[7:]
            data = m.parse(data)
            source_text2.append(data)
        else:
            data = row[8:]
            data = m.parse(data)
            target_text2.append(data)

In [13]:
source_text2 = source_text2[:10000]
target_text2 = target_text2[:10000]

## Preprocess

In [105]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    eos = target_vocab_to_int['<EOS>']
    source_id_text = [[source_vocab_to_int[word] for word in sequence.split()] 
                      for sequence in source_text]
    target_id_text = [[target_vocab_to_int[word] for word in sequence.split()] + [eos] 
                      for sequence in target_text]
    return source_id_text, target_id_text

In [106]:
CODES = {'<PAD>': 0, '<EOS>': 1, '<UNK>': 2, '<GO>': 3 }

In [107]:
import copy
def create_lookup_tables(text):
    vocab = set()
    for line in text:
        for word in line.split(" "):
                vocab.add(word)
    
    vocab_to_int = copy.copy(CODES)

    for v_i, v in enumerate(vocab, len(CODES)):
        vocab_to_int[v] = v_i

    int_to_vocab = {v_i: v for v, v_i in vocab_to_int.items()}

    return vocab_to_int, int_to_vocab

In [115]:
source_vocab_to_int, source_int_to_vocab = create_lookup_tables(source_text2)
target_vocab_to_int, target_int_to_vocab = create_lookup_tables(target_text2)

source_int_text, target_int_text = text_to_ids(source_text2, target_text2, 
                                               source_vocab_to_int, target_vocab_to_int)

In [116]:
len(source_vocab_to_int), len(target_vocab_to_int)

(7014, 6178)

In [66]:
# Save Data
with open('preprocess.p', 'wb') as out_file:
    pickle.dump((
        (source_text2, target_text2),
        (source_vocab_to_int, target_vocab_to_int),
        (source_int_to_vocab, target_int_to_vocab)), out_file)

In [67]:
def load_preprocess():
    with open('preprocess.p', mode='rb') as in_file:
        return pickle.load(in_file)

## Build the Neural Network

In [21]:
import tensorflow as tf

In [22]:
def model_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name="input")
    targets = tf.placeholder(tf.int32, [None, None], name="target")
    learning_rate = tf.placeholder(tf.float32, name="learning_rate")
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    target_seq_length = tf.placeholder(tf.int32, [None], name="target_sequence_length")
    max_target_len = tf.reduce_max(target_seq_length, name="max_target_len")
    src_seqence_length = tf.placeholder(tf.int32, [None], name="source_sequence_length")
    return inputs, targets, learning_rate, keep_prob, target_seq_length, max_target_len, src_seqence_length

In [37]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    go = target_vocab_to_int['<GO>']
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], go), ending], 1)
    return dec_input

In [24]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):

    enc_embed_input = tf.contrib.layers.embed_sequence(
        rnn_inputs, source_vocab_size, encoding_embedding_size)

    def make_cell(rnn_size):
        enc_cell = tf.contrib.rnn.LSTMCell(
            rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        enc_cell = tf.contrib.rnn.DropoutWrapper(enc_cell, keep_prob)
        return enc_cell

    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    
    enc_output, enc_state = tf.nn.dynamic_rnn(
        enc_cell, enc_embed_input, sequence_length=source_sequence_length, dtype=tf.float32)
    return enc_output, enc_state

In [25]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    training_helper = tf.contrib.seq2seq.TrainingHelper(
        inputs=dec_embed_input, sequence_length=target_sequence_length, time_major=False)

    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                                       training_helper,
                                                       encoder_state, output_layer)

    training_decoder_output, _ = tf.contrib.seq2seq.dynamic_decode(
        training_decoder, impute_finished=True,
        maximum_iterations=max_summary_length)

    logits = training_decoder_output.rnn_output
    sample_id = training_decoder_output.sample_id
    logits = tf.nn.dropout(logits, keep_prob)
    
    return tf.contrib.seq2seq.BasicDecoderOutput(logits, sample_id)

In [27]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    start_tokens = tf.tile(
        tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], 
        name='start_tokens')

    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
        dec_embeddings, start_tokens, end_of_sequence_id)

    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        encoder_state,
                                                        output_layer)
        
    inference_decoder_output, _ = tf.contrib.seq2seq.dynamic_decode(
        inference_decoder, impute_finished=True,
        maximum_iterations=max_target_sequence_length)

    logits = inference_decoder_output.rnn_output
    sample_id = inference_decoder_output.sample_id
    logits = tf.nn.dropout(logits, keep_prob)
    
    return tf.contrib.seq2seq.BasicDecoderOutput(logits, sample_id)

In [39]:
from tensorflow.python.layers.core import Dense
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    # 1. Embed the target sequences
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size,
                                                    decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    # 2. Construct the decoder LSTM cell
    def make_cell(rnn_size):
        lstm = tf.contrib.rnn.LSTMCell(
            rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        dropout = tf.contrib.rnn.DropoutWrapper(lstm, keep_prob)
        return dropout
    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])

    # 3. Create an output layer 
    output_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0,
                                                                              stddev=0.1))
    
    # 4. Use the your decoding_layer_train
    with tf.variable_scope("decode"):
        train_logits = decoding_layer_train(encoder_state, 
                                            dec_cell, 
                                            dec_embed_input, 
                                            target_sequence_length,
                                            max_target_sequence_length, 
                                            output_layer, 
                                            keep_prob) 
    
    # 5. Use the your decoding_layer_infer
    with tf.variable_scope("decode", reuse=True):
        infer_logits = decoding_layer_infer(encoder_state, 
                                            dec_cell, 
                                            dec_embeddings, 
                                            target_vocab_to_int['<GO>'], 
                                            target_vocab_to_int['<EOS>'],
                                            max_target_sequence_length, 
                                            target_vocab_size,
                                            output_layer, 
                                            batch_size,
                                            keep_prob)

    return train_logits, infer_logits

In [35]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    # TODO: Implement Function
    _, enc_state = encoding_layer(input_data, 
                               rnn_size, 
                               num_layers,
                               keep_prob,
                               source_sequence_length,
                               source_vocab_size, 
                               enc_embedding_size)
    
    
    # Prepare the target sequences we'll feed to the decoder in training mode
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    # Pass encoder state and decoder inputs to the decoders
    training_decoder_output, inference_decoder_output = decoding_layer(
        dec_input, 
        enc_state, 
        target_sequence_length,
        max_target_sentence_length, 
        rnn_size, 
        num_layers,
        target_vocab_to_int,
        target_vocab_size, 
        batch_size,
        keep_prob, 
        dec_embedding_size)
    
    return training_decoder_output, inference_decoder_output

## Neural Network Training

In [111]:
# Number of Epochs
epochs = 1
# Batch Size
batch_size = 100
# RNN Size
rnn_size = 64
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 128
decoding_embedding_size = 128
# Learning Rate
learning_rate = 0.01
# Dropout Keep Probability
keep_probability = 0.5
display_step = 8

In [112]:
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = load_preprocess()

max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

In [113]:
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]

def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths

In [117]:
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    8/1000 - Train Accuracy: 0.5968, Validation Accuracy: 0.8013, Loss: 5.6700
Epoch   0 Batch   16/1000 - Train Accuracy: 0.5476, Validation Accuracy: 0.8013, Loss: 5.9001
Epoch   0 Batch   24/1000 - Train Accuracy: 0.6559, Validation Accuracy: 0.8063, Loss: 5.7980
Epoch   0 Batch   32/1000 - Train Accuracy: 0.5600, Validation Accuracy: 0.8051, Loss: 6.2951
Epoch   0 Batch   40/1000 - Train Accuracy: 0.4667, Validation Accuracy: 0.8038, Loss: 5.4763
Epoch   0 Batch   48/1000 - Train Accuracy: 0.4688, Validation Accuracy: 0.8013, Loss: 5.7669
Epoch   0 Batch   56/1000 - Train Accuracy: 0.0242, Validation Accuracy: 0.0165, Loss: 4.9401
Epoch   0 Batch   64/1000 - Train Accuracy: 0.0526, Validation Accuracy: 0.0165, Loss: 5.9094
Epoch   0 Batch   72/1000 - Train Accuracy: 0.7656, Validation Accuracy: 0.8025, Loss: 4.6099
Epoch   0 Batch   80/1000 - Train Accuracy: 0.4917, Validation Accuracy: 0.8063, Loss: 5.6332
Epoch   0 Batch   88/1000 - Train Accuracy: 0.0333, Validati

KeyboardInterrupt: 

In [118]:
import pickle
def save_params(params):
    with open('params.p', 'wb') as out_file:
        pickle.dump(params, out_file)

def load_params():
    with open('params.p', mode='rb') as in_file:
        return pickle.load(in_file)

In [119]:
save_params(save_path)

## Predict

In [122]:
import tensorflow as tf
import numpy as np

load_path = load_params()

In [123]:
def sentence_to_seq(sentence, vocab_to_int):
    m = MeCab.Tagger("-Owakati")
    unk = vocab_to_int['<UNK>']
    sentence = m.parse(sentence)
    return [vocab_to_int.get(w, unk) for w in sentence.split()]

In [124]:
translate_sentence = 'こんにちは。暑いね'

translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))

INFO:tensorflow:Restoring parameters from checkpoints/dev


InvalidArgumentError: indices[0,4] = 6833 is not in [0, 6524)
	 [[Node: EmbedSequence/embedding_lookup = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@EmbedSequence/embeddings"], validate_indices=true, _device="/job:localhost/replica:0/task:0/cpu:0"](EmbedSequence/embeddings/read, ReverseV2)]]

Caused by op 'EmbedSequence/embedding_lookup', defined at:
  File "/home/tsu-nera/anaconda3/envs/dlnd/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/tsu-nera/anaconda3/envs/dlnd/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/tsu-nera/anaconda3/envs/dlnd/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/tsu-nera/anaconda3/envs/dlnd/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/tsu-nera/anaconda3/envs/dlnd/lib/python3.6/site-packages/ipykernel/kernelapp.p

In [125]:
def user_input(sentence):
    sentence = sentence_to_seq(sentence, source_vocab_to_int)
    print(sentence)

    loaded_graph = tf.Graph()
    with tf.Session(graph=loaded_graph) as sess:
        # Load saved model
        loader = tf.train.import_meta_graph(load_path + '.meta')
        loader.restore(sess, load_path)

        input_data = loaded_graph.get_tensor_by_name('input:0')
        logits = loaded_graph.get_tensor_by_name('logits:0')
        keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

        translate_logits = sess.run(logits, {input_data: [sentence], keep_prob: 1.0})[0]
        
    return ''.join([target_int_to_vocab[i] for i in np.argmax(translate_logits, 1)])

In [126]:
user_input("夏といえば海だね")

[5300, 5719, 5700, 6236, 4830, 1078, 2809]
INFO:tensorflow:Restoring parameters from checkpoints/dev


'<PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>'

In [ ]:
user_input("君は泳げるのかい？")

In [ ]:
user_input("海といえば、なんですか？")

In [ ]:
user_input("山はどうだい？")

In [ ]:
user_input("こんにちは")

In [ ]:
user_input("元気ですか？暑くなって来ましたね？")

In [ ]:
user_input("バカ")

In [ ]:
user_input("君はバカ？")

In [ ]:
user_input("今はどこに住んでいるのかな？")

In [ ]:
user_input("オリンピックが東京で開催されるの知ってる？")

In [ ]:
user_input("海はいいです")